In [1]:
import json
import numpy as np
#import matplotlib
import pandas as pd
import geopandas as gpd
import sqlite3
from shapely.geometry import Point

In [2]:
def ray_tracing_numpy(x,y,poly):
    n = len(poly)
    inside = np.zeros(len(x),np.bool_)
    p2x = 0.0
    p2y = 0.0
    xints = 0.0
    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        idx = np.nonzero((y > min(p1y,p2y)) & (y <= max(p1y,p2y)) & (x <= max(p1x,p2x)))[0]
        if p1y != p2y:
            xints = (y[idx]-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
        if p1x == p2x:
            inside[idx] = ~inside[idx]
        else:
            idxx = idx[x[idx] <= xints]
            inside[idxx] = ~inside[idxx]    

        p1x,p1y = p2x,p2y
    return inside

In [3]:
vels = pd.read_csv('../geod/e_asia_vels_eur.csv')

In [53]:
#blocks = gpd.read_file('../block_data/chn_faults_blocks.gpkg', layer='blocks')
blocks = gpd.read_file('../block_data/chn_blocks.geojson')

In [54]:
blocks.columns

Index(['fid', 'geometry'], dtype='object')

In [55]:
blocks.set_index('fid')

,geometry
fid,
2,"POLYGON ((77.38402 36.46123, 77.50081 36.44633..."
3,"POLYGON ((77.83795 34.30176, 78.08587 34.30104..."
4,"POLYGON ((82.64719 30.96322, 82.24315 30.73499..."
5,"POLYGON ((81.68896 30.76847, 81.56945 30.60804..."
6,"POLYGON ((80.13940 34.83448, 80.54500 34.56131..."
7,"POLYGON ((82.64719 30.96322, 82.96203 30.87999..."
8,"POLYGON ((84.35318 30.65382, 84.39520 30.67245..."
9,"POLYGON ((82.24315 30.73499, 82.64719 30.96322..."
10,"POLYGON ((83.70032 31.30484, 83.91885 31.16050..."


In [56]:
def _get_pt_geom(row):
    return Point(row.lon, row.lat)

In [57]:
vels['geometry'] = vels.apply(_get_pt_geom, axis=1)

In [58]:
crs = "epsg:4326"

In [59]:
vels_gdf = gpd.GeoDataFrame(vels, crs=crs)

In [60]:
vels_gdf

,lon,lat,e_vel,n_vel,e_err,n_err,station,ref,geometry
0,105.815,33.340,7.300,-1.800,0.800,0.700,1375_GPS,wang_shen_2020,POINT (105.81500 33.34000)
1,112.614,41.271,3.000,-0.900,0.300,0.300,A001_GPS,wang_shen_2020,POINT (112.61400 41.27100)
2,112.564,40.892,3.200,-1.300,0.200,0.200,A002_GPS,wang_shen_2020,POINT (112.56400 40.89200)
3,112.480,40.529,3.500,-1.300,0.200,0.200,A003_GPS,wang_shen_2020,POINT (112.48000 40.52900)
4,112.354,40.171,4.200,-1.700,0.200,0.200,A004_GPS,wang_shen_2020,POINT (112.35400 40.17100)
5,113.132,41.019,3.400,-1.100,0.200,0.200,A005_GPS,wang_shen_2020,POINT (113.13200 41.01900)
6,113.213,40.787,3.200,-1.300,0.300,0.200,A006_GPS,wang_shen_2020,POINT (113.21300 40.78700)
7,113.196,40.447,3.900,-1.600,0.200,0.200,A007_GPS,wang_shen_2020,POINT (113.19600 40.44700)
8,112.727,40.016,4.800,-1.900,0.300,0.300,A008_GPS,wang_shen_2020,POINT (112.72700 40.01600)
9,113.887,40.876,2.900,-1.200,0.200,0.200,A011_GPS,wang_shen_2020,POINT (113.88700 40.87600)


In [61]:
blk_idx = gpd.sjoin(vels_gdf, blocks, how='inner').fid

In [62]:
in_vels = vels.loc[blk_idx.index]

In [63]:
in_vels['block'] = blk_idx

In [64]:
in_vels

,lon,lat,e_vel,n_vel,e_err,n_err,station,ref,geometry,block
136,85.358,27.697,9.300,35.800,0.900,0.700,AIRP_GPS,wang_shen_2020,POINT (85.35800 27.69700),19
1733,85.521,27.693,9.600,34.500,0.200,0.200,NAGA_GPS,wang_shen_2020,POINT (85.52100 27.69300),19
2243,85.314,28.207,9.600,28.800,0.200,0.200,CHLM_GPS,wang_shen_2020,POINT (85.31400 28.20700),19
2251,85.279,27.801,9.600,34.500,0.200,0.200,KKN4_GPS,wang_shen_2020,POINT (85.27900 27.80100),19
2259,83.936,28.260,8.900,32.300,0.400,0.400,SRNK_GPS,wang_shen_2020,POINT (83.93600 28.26000),19
2264,85.799,27.385,11.100,35.300,0.300,0.300,SNDL_GPS,wang_shen_2020,POINT (85.79900 27.38500),19
2333,83.978,28.199,8.400,34.100,0.500,0.400,POKH_GPS,wang_shen_2020,POINT (83.97800 28.19900),19
2512,85.008,27.316,9.503,37.446,1.000,0.900,HET0,Bettinelli_JOG_2006,POINT (85.00800 27.31600),19
2518,85.398,27.575,11.685,36.174,1.000,0.900,PKI0,Bettinelli_JOG_2006,POINT (85.39800 27.57500),19
2519,85.222,28.015,11.431,30.015,1.000,0.900,RAM0,Bettinelli_JOG_2006,POINT (85.22200 28.01500),19


In [65]:
in_vels[['lon', 'lat', 'e_vel', 'n_vel', 'e_err', 'n_err', 'station', 'block']].to_csv('../block_data/block_vels.csv', index=False)